In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as ss
import pickle

from corextopic import corextopic as ct
from corextopic import vis_topic as vt

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
ar_df = pickle.load(open("ar_df_3mo.pkl", 'rb'))

In [5]:
ar_df.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to,text,lang
34138,1311516689650122752,1311516689650122752,2020-09-30 23:02:00 CDT,2020-09-30,23:02:00,-600,1174210043664838657,faoinuae,"FAO for GCC states, UAE & Yemen",NaN,...,"['سبل_عيش', 'كوفيد_19']",[],https://twitter.com/FAOinUAE/status/1311516689...,False,NaN,0,NaN,[],هل تعلم أن الأسماك 🐟🐠 تعد واحدة من السلع الغذا...,ar
34146,1311507758160777216,1311507758160777216,2020-09-30 22:26:30 CDT,2020-09-30,22:26:30,-600,951917493442015232,addafternet,منصة الدفتر العلمية ®️,NaN,...,[],[],https://twitter.com/addafternet/status/1311507...,False,NaN,0,NaN,[],عمليات الإغلاق المترافقة مع انتشار فيروس كوفيد...,ar
34147,1311506585508773889,1311360242521985024,2020-09-30 22:21:51 CDT,2020-09-30,22:21:51,-600,718412478,eddy_gemayel,Eddy Gemayel,NaN,...,[],[],https://twitter.com/eddy_gemayel/status/131150...,False,NaN,0,NaN,"[{'screen_name': 'issahamd1n', 'name': 'Issa H...",الاطباء هم الأكثر عرضةً للهجرة أثناء أزمة ا...,ar
34148,1311505444657983489,1311505444657983489,2020-09-30 22:17:19 CDT,2020-09-30,22:17:19,-600,1017445720956198912,liallal_develop,liallal,NaN,...,"['سكاي_رياضه', 'ايران', 'كوفيد_19', 'سيف_بن_زا...",[],https://twitter.com/liallal_develop/status/131...,False,NaN,0,NaN,[],I will do organic youtube video spotify music...,ar
34149,1311505387447705600,1311505387447705600,2020-09-30 22:17:05 CDT,2020-09-30,22:17:05,-600,1017445720956198912,liallal_develop,liallal,NaN,...,"['سكاي_رياضه', 'ايران', 'كوفيد_19', 'سيف_بن_زا...",[],https://twitter.com/liallal_develop/status/131...,False,NaN,0,NaN,[],I will do organic promotion youtube video ...,ar


In [41]:
en_df.filtered = en_df.filtered.apply(lambda x: ' '.join(x))

In [44]:
en_df.filtered.head()

119134    She Khaleda Begum age years saved Rs lakh perf...
119140    ImranKhanPTI ClarenceHouse BorisJohnson koi re...
119128                                 sylver Corona ye Ref
119111    The new music post quarantine finna repetitive...
119113    My dad high risk Corona bc health amp I ’ work...
Name: filtered, dtype: object

In [6]:
ar_df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'reply_to', 'text', 'lang'],
      dtype='object')

In [7]:
ar_df.lang.value_counts()

ar    288876
Name: lang, dtype: int64

In [9]:
ar_dtm = pickle.load(open("tfidf_ar_sparse_3mo.pkl", 'rb'))

In [8]:
ar_vectorizer = pickle.load(open("ar_vectorizer_3mo.pkl", 'rb'))

In [10]:
words = list(np.asarray(ar_vectorizer.get_feature_names()))

In [12]:
topic_model = ct.Corex(n_hidden=30, words=words, seed=55)
topic_model.fit(ar_dtm, words=words, docs=ar_df.tweet)

### What Are My Topics?

In [13]:
topics = topic_model.get_topics()
for n, topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: ال,co,الله,اللي,ات,20,مش,لقاح,عشان,19
1: حال,وفا,تسجيل,جديد,إصاب,24,إجمالي,ساع,شفاء,ماضي
2: اخت,the,شقيق,حافي,بيوإنتك,ممزق,بروفسور,أوغور,صغرى,ازياء
3: مملكه,وطني_أمن_للإنسانية,aou_ksab,مجان,أوائل_الإمارات,ماسبق,أنشطه,مبادر,تبرع,منص
4: عبل,عنتر,الفاينل_عن_بعد,جامعه_العين,aau_uae,3meed_news,uae_barq,امه_2020,امة_2020,الرشايدة_نيوز
5: صباح_الخير,النصر_الباطن_الدوري,الأقتران_العظيم,سناب,الشتاء_حولك,الفرق_بين_البدو_والحضر,مساء_الخير,السلالة_الجديدة,المشروب_الرسمي_للشتاء,مقاطعه_المنتجات_الفرنسيه55
6: يدينك,زندق,صامته,تعقم,مفاتيح,مادم,نتقيد,خوفا,فوا,نسق
7: يماني,ناصر,إمام,مهدي,إخوان,الصحة_النفسية,غانم_الحساني,l0tzp8mzep,1441,هزل
8: الوقائع,الوقائع_الاخبارية,alwakaai,وقائع,الاردن,احتياطات_موجه_كورونا,اتحاد_مغردي_السودان,مدونه_الاتحاد,sudtwittun,حظر_تجول
9: سيتي,فولهام,مانشستر,إيفرتون,توتنهام,مبارا,تأجيل,تيليس,أليكس,ايفرتون
10: صحتك_أهم,الاسر_المنتجة,افكار_تجارية,w81y3lsayp,طنج,sahtak,لاتتردد,أوكسيجين,طوفار,صوير
11: distancing,social,أجتماعي,أسلم,نقانق,شطائر,هودي,قاطعوهم,تتساهل,الحكاية
12: أ

### Top Tweets Associated With A Topic

In [ ]:
topic_model.get_top_docs(topic=4, n_docs=2)

### Which Topics Has CorEx Assigned Each Document?

In [16]:
anchor_words = [['سارين'], 
                ['أعشاب', 'ثوم', 'ليمون', 'شفاء','فيتامين'], 
                ['اختراع'], 
                ['ناموس', 'بعوض'], 
                ['بيل', 'مشروع',' جيتس'], 
                ['كهرومغناطيسية'], 
                ['5g','ID','2020'], 
                ['نباتي'], 
                ['أنفلونزا'], 
                ['بيولوجي', 'حرب']]

In [36]:
anchors_combo = ['سارين', 
                'أعشاب', 'ثوم', 'ليمون', 'شفاء','فيتامين', 
                'اختراع', 
                'ناموس', 'بعوض', 
                'بيل', 'مشروع',' جيتس', 
                'كهرومغناطيسية', 
                ' 5g ','ID','2020', 
                'نباتي', 
                'أنفلونزا', 
                'بيولوجي', 'حرب']

In [37]:
an_ar_topic_model = ct.Corex(n_hidden=30, max_iter=500, words=words, seed=55)
an_ar_topic_model.fit(ar_dtm, words=words, anchors=anchors_combo, anchor_strength=10, docs=ar_df.tweet)

In [38]:
an_topics = an_ar_topic_model.get_topics()
for n, topic in enumerate(an_topics):
    topic_words,_ = zip(*an_ar_topic_model.get_topics(topic=n))
    print('{}: '.format(n) + ','.join(topic_words))

0: أعشاب,صحيفة_الشرق,إنفوجرافيك,مجلة_الفجيرة,fujmagazine,fujairah_magazine,المملكة,نعود_بحذر,الفجيرة,يحظر
1: ثوم,تجاري,مستوفي,محل,مخالف,تدابير,منشآت,احترازي,منشأ,تنبيه
2: ليمون,الرشايدة_نيوز,امه_2020,امة_2020,kuwait,لا_للأختبارات_الورقيه_كورونا_قاتل,عيالنا_مو_لعبه_بإيدكم,elkhabar,إلكتر,moekuwait
3: شفاء,حال,تسجيل,إجمالي,جديد,24,ساع,وفا,إصاب,ماضي
4: فيتامين,اتحاد_مغردي_السودان,مدونه_الاتحاد,sudtwittun,لميس_الحديدي,كلمة_أخيرة,احتياطات_موجه_كورونا,lameesh,نقص,on
5: اختراع,حماي,ينته,البس_كمامة,كحولي,wearamask,مطهر,صابون,إستمرار,حافظ
6: رقمي,جذور,تجلى,سناب,صباح_الخير,النصر_الباطن_الدوري,تحتي,الأقتران_العظيم,السلالة_الجديدة,متين
7: بعوض,جهاز_التخطيط_والاحصاء,خليك_على_بال,أشطاري24,yoursafetyismysafety,إضغط,بشام,دجمبر,مفحوص,w81y3lsayp
8: بيل,غيتس,issh75syvh,قرصن,منبثق,جيتس,فيس,بوك,مايكروسوفت,منشور
9: مشروع,أوائل_الإمارات,وسم,نموذج,رشح,forsan_uae,أرشح,الصحة_النفسية,غانم_الحساني,شخصي
10: كهرومغناطيسية,احترار,غرد_بصورة,اسواق_آسيا,ترفد,كاوس,فنلندي,bualiben,لجم,579ku
11: 2020,يماني,تاريخ,11,ناصر,29

In [21]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/jess/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
stop_words_std = stopwords.words('arabic')
stopword_url = "/Users/jess/workspace/Metis_Projects/Project_5/camel_tools/arabic-stop-words/list.txt"

In [23]:
def get_stopwords(file_location):
    lines = []
    file = open(file_location)
    lines = file.read().split('\n')
        
    file.close()
    return lines

stopwords_long = get_stopwords(stopword_url)

In [24]:
my_stop_words = set(stop_words_std + stopwords_long + ['http', 'https'])

In [28]:
cv = CountVectorizer(lowercase=True,
                     stop_words=my_stop_words,
                     max_df = 0.8,
                     min_df = 2,
                     ngram_range=(1,2)
                    )

In [29]:
corex_vector = cv.fit_transform(ar_df.tweet)

In [34]:
cv_topic_model = ct.Corex(n_hidden=30, max_iter=3000, seed=55)
cv_topic_model.fit(corex_vector, anchors=anchor_words, anchor_strength=10, words=cv.get_feature_names())

In [33]:
cv_topics = cv_topic_model.get_topics()
for n, topic in enumerate(cv_topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: شفاء,جديدة,إصابة,إصابة جديدة,حالة,وفاة,جديدة بفيروس,تسجيل,بفيروس كورونا,بفيروس
1: اختراع,المكلفة,المكلفة ببحث,ببحث,ببحث آلية,العليا المكلفة,آلية التعامل,التطورات الناتجة,التعامل التطورات,الناتجة
2: أوغور,شاهين,أوغور شاهين,مؤسس شركة,شاهين مؤسس,مؤسس,حافي القدمين,بيوإنتك,حافي,شركة بيوإنتك
3: مشروع,بيل,المجتمع,يخضعون للحجر,للحجر الصحي,الخارج يخضعون,يخضعون,محلية مسجلة,مسجلة أفراد,الأخيرة بينها
4: جائحة,جائحة كوفيد,الاحترازية,الإجراءات,الصحية,جائحة كورونا,للحد,القطاع,الوقائية,التعليم
5: 2020,11,11 2020,محمد اليماني,اليماني,al,الإمام,ناصر محمد,menafn al,2020 co
6: أنفلونزا,فايروس حل,المتحدة وكادت,وكادت تنهار,حل بالعراق,تنهار بفعل,بالعراق امريكا_تنتفض,وكادت,حل حل,بفعل فايروس
7: حرب,بالتدابير الاحترازية,بالتدابير,المستوفية,الالتزام بالتدابير,بيولوجي,لعدم الالتزام,عالية الجودة,اقتصادية دبي,محلا
8: تطبيق,عبر تطبيق,نبض,nabdapp,للجميع,تطبيق نبض,nabdapp co,نبض nabdapp,مركز,مساعدات
9: لبنان,coronavirus,غزة,coronavirus co,مصر,إيران,لمدة,قطاع غزة,العراق,قطاع
10: العالم,الفيروس,الدول,سلالة,الوباء,دول

In [49]:
w2v_anchors = [['plandemic', 'scamdemic', 'fearmongering', 'coronavirushoax', 'covidhoax', 'fabricated', 'smokescreen']]

In [32]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/jess/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
stop_words_standard = stopwords.words('english')
stop_words_long = open('/Users/jess/workspace/Metis_Projects/Project_4/stopwords/stopwords_long_ranks_nl.txt', 'r', encoding='utf-8').read().split('\n')
stop = set(stop_words_standard + stop_words_long + ['https' , 'http'])

In [37]:
texts = [[word.lower() for word in tweet if word not in stop] for tweet in en_df.tokens.values]
print(texts[0:5])

[['she', 'khaleda', 'begum', 'age', 'years', 'saved', 'rs', 'lakh', 'perform', 'hajj', 'donated', 'corona', 'charity', '❤'], ['imrankhanpti', 'clarencehouse', 'borisjohnson', 'koi', 'relief', 'nhi', 'mila', 'relief', 'granted', 'likha', 'ata', 'hy', 'dfa', 'complaint', 'ki', 'citizen', 'portal', 'complaint', 'cell', 'ki', 'post', 'office', 'calls', 'ki', 'kia', 'faida', 'hua', 'apka', 'itny', 'pese', 'lgany', 'ka', 'agr', 'aj', 'doctr', 'ali', 'hassan', 'awan', 'ko', 'mri', 'complaint', 'koi', 'saza', 'di', 'gai', 'hti', 'relief', 'dia', 'hta', 'corona'], ['sylver', 'corona', 'ye', 'ref'], ['the', 'new', 'music', 'post', 'quarantine', 'finna', 'repetitive', 'bet', '’', 'hear', 'countless', 'times', 'she', 'loner', 'loan', 'uh', 'i', 'kill', 'pussy', 'corona', 'the', 'rap', 'game', 'mine', 'i', 'murder', 'corona', 'columbine', '🥴🥴🥴'], ['my', 'dad', 'high', 'risk', 'corona', 'bc', 'health', 'amp', 'i', '’', 'working', 'amp', 'high', 'risk', 'transferring', 'i', '’', 'visit', 'parents', '